# Youtube Recommendation Video System

In [0]:
# Import Libraries
import pandas as pd
import numpy as np
import re
import time
import bs4 as bs4
import requests as rq
import json
import os 


 ## Data Search


* #### Webscraping youtube





## 1.  Youtube search page Structure (URL)




The image below shows the strucute of the youtube url. We have the main page, then the search page(opens when we search our query), the way the videos will be sorted ( you define it by clicking on Filter) and finally the number of the page that's being searched. 


![alt text](https://drive.google.com/uc?id=15I0wM3OOiHQ3J3Ls_dDoq4Fax9PILFAw)



In [0]:
# what I want to search 
queries = ['machine+learnin','data+science','deploy' ] 
# The url structure of youtube - see image below
url = "http://www.youtube.com/results?search_query={query}&sp=CAI%253D&p={page}"

In [31]:
# example of a search query 
url.format(query ="machine+learning",page = 1)

'http://www.youtube.com/results?search_query=machine+learning&sp=CAI%253D&p=1'

In [0]:
# Making sure the folder I will store the raw data exists (create directory in case it does not exist)
data_dir = '/content/drive/My Drive/Colab Notebooks/Youtube_Video_Recommendation_System/raw_data'
if not os.path.exists(data_dir):
  os.makedirs(data_dir)

In [0]:
for query in queries:
  for page in range(1,101): #from page 0 to 100
    urll = url.format(query = query, page= page)
    print(urll)
    response = rq.get(urll) # get request to retrieve data from page

    # Save each url in raw_data folder 
    with open("/content/drive/My Drive/Colab Notebooks/Youtube_Video_Recommendation_System/raw_data/{}_{}.html".format(query,page),"w+") as output:
      output.write(response.text)
      time.sleep(2)



# Data Parsing

Now that I have all urls saved in my raw_data folder, I am able to process and  each of them the format I want

###  Test data Parsing 

In [0]:
for query in queries[:1]:
  for page in range(1,2):
    # read each page 
    with open("/content/drive/My Drive/Colab Notebooks/Youtube_Video_Recommendation_System/raw_data/{}_{}.html".format(query, page), "r+") as f: 
      page_html = f.read()

      # BeautifulSoup Object to be parsed
      parsed = bs4.BeautifulSoup(page_html)

      # The tag we find the information we want
      # The tag "a" defines a hyperlink, which is used to link from one page to another
      tags = parsed.findAll("a")

      for e in tags:
        if e.has_attr("aria-describedby"):
          link = e['href']      # get link addres 
          title = e['title']    # get link title

          # Save parsed as json file 
          with open("parsed_videos.json", "a+") as output:
            data = {"link": link, 
                    "title":title,
                    "query":query}

            output.write("{}\n".format(json.dumps(data)))



In [46]:
# Check if my json file is the way I want
print('Parsed object type {}:'.format(type(parsed)))
print()
print('First element (first page link):\n{}'.format(tags[0]))
print()
print('Classes of the tag:\n{}'.format(tags[0]['class']))
print()
print('The json file:\n{}'.format(json.dumps(data)))

Parsed object type <class 'bs4.BeautifulSoup'>:

First element (first page link):
<a class="masthead-logo-renderer yt-uix-sessionlink spf-link " data-sessionlink="itct=CAMQsV4iEwjo4YKcqtbnAhUhm8IKHUWDDdU" href="/" id="logo-container" title="YouTube Home"> <span class="logo masthead-logo-renderer-logo yt-sprite" title="YouTube Home"></span>
</a>

Classes of the tag:
['masthead-logo-renderer', 'yt-uix-sessionlink', 'spf-link', '']

The json file:
{"link": "/watch?v=0qSatLhCKxg", "title": "Gregory Horror Show PS2 FMV 7 Death Introduction (AI Machine Learning Upscaling)", "query": "machine+learnin"}


# Verify Results

In [50]:
df = pd.read_json("parsed_videos.json",lines = True)
df.head()

,link,title,query
0,/watch?v=OlHpaDSPA4w,how to be creative and teach yourself jazz (as...,machine+learnin
1,/watch?v=kG0TjlayW64,Faire du Machine Learning plus vite qu'un expert,machine+learnin
2,/watch?v=jSfSqJA6_3k,Machine Learning: A Beginner's Introduction to...,machine+learnin
3,/watch?v=F0gcpZmtx3g,Self learning diary - Machine Learning - Day 1,machine+learnin
4,/watch?v=VUEYBhVDSC0,Bias and variance trade off in machine learnin...,machine+learnin
